## load 参数

In [4]:
import numpy as np
import cv2
import scipy.io as scio

Params=scio.loadmat('workspace.mat')
K = np.array(Params['K'])
K = K.T
distortion=np.array(Params['distortion'])

## sift特征匹配

In [5]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

imgpath = 'images/'
MIN_MATCH_COUNT = 10

def match_pts(imgname1, imgname2):
    img1 = cv2.imread(imgpath+imgname1, 0)          # queryImage
    img2 = cv2.imread(imgpath+imgname2, 0)        # trainImage

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    pts1 = []
    pts2 = []
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    # return the np.array type
    return pts1, pts2

# estimate估计参数

In [40]:
from numpy import *
from scipy import *
from scipy.linalg import *
from pylab import *
import cv2
import sys

def find_fundamental(pts1, pts2):
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    return F
    
def find_essential(F, K1, K2):
    return K2.T @ F @ K1

def skew(a):
    #Skew matrix A such that cross(a,v) = Av for any v
    return array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])

def anti_skew(A):
    return array([A[2,1], A[2,0], A[1,0]])

def triangulate(P1, P2, pts1, pts2):
    n1 = pts1.shape[0]
    n2 = pts2.shape[0]
    
    p1 = vstack([ pts1.T, ones(n1) ]).T
    p2 = vstack([ pts2.T, ones(n2) ]).T
    
    P = zeros((n1, 4))
    for i in range(n1):
        cross1 = skew(p1[i])
        cross2 = skew(p2[i])
        Q = vstack([cross1 @ P1, cross2 @ P2])
        U,S,VT = svd(Q)
        z = VT[-1, :]
        P[i, :] = z/z[3]
    
    return P[:, 0:3]

def guess_rotation(E):
    # make sure E is rank 2
    U,S,V = svd(E)
    if det(dot(U,V)) < 0:
        V = -V
    E = dot(U,dot(diag([1,1,0]),V))    

    W = array([[0,-1,0],[1,0,0],[0,0,1]])
        
    R21 = dot(U,dot(W,V))
    R22 = dot(U,dot(W.T,V))
    
    T21 = anti_skew(E @ R21.T) 
    T22 = anti_skew(E @ R22.T)
    
    return R21, T21, R22, T22

def test_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    for v in temp:
        if v[-1] < 0:
            return False
    return True
    

def find_rotation(E, K1, K2, pts1, pts2):
    R1 = eye(3)
    T1 = array([0, 0, 0])
    P1 = K1 @ vstack([R1.T, T1]).T
    
    R21, T21, R22, T22 = guess_rotation(E)
    
    P21 = K2 @ vstack([R21.T, T21]).T
    P22 = K2 @ vstack([R22.T, T22]).T

    key1 = test_rotation(P1, P21, pts1, pts2)
    key2 = test_rotation(P1, P22, pts1, pts2)
    
    R2 = R21
    T2 = T21

    if key1:
        R2 = R21
        T2 = T21
    elif key2 or True:
        R2 = R22
        T2 = T22
    else:
        print('Can not find the right R matrix.')
        sys.exit()

    return R1, T1, R2, T2
    
def find_pose(R1, T1, R2, T2, K1, K2):
    #return K @ vstack([R.T, T]).T
    P1 = K1 @ vstack([R1.T, T1]).T
    P2 = K2 @ vstack([R2.T, T2]).T
    
    return P1, P2


## 运行

In [7]:
pts1, pts2 = match_pts('1_.jpg', '2_.jpg')
print(pts1.shape)

(1340, 2)


In [8]:
F = find_fundamental(pts1, pts2)
E = find_essential(F, K, K)
print('ok')

ok


In [41]:
R1, T1, R2, T2 = find_rotation(E, K, K, pts1, pts2)
print('ok')

ok


In [49]:
print(R1, T1, R2, T2, sep='\n')
K1 = K
K2 = K
P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
points = triangulate(P1, P2, pts1, pts2)
print('the space points:', points[500:510], points[300:310], sep ='\n---------\n')

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0 0 0]
[[ 0.99680378 -0.01337628  0.07876103]
 [ 0.01358739  0.99990539 -0.00214501]
 [-0.07872489  0.00320831  0.99689122]]
[-0.9751416   0.00491469  0.22152812]
the space points:
---------
[[-8.53211717e-01  8.15531216e-01  1.10123780e+01]
 [-1.16630696e+01 -6.23401153e+01  1.04358005e+02]
 [-1.24331761e+00 -1.77197804e-01  1.40915108e+01]
 [-3.95591507e+00 -2.02841129e+01  3.81129268e+01]
 [ 5.69331577e+00  3.03069102e+01 -4.56453265e+01]
 [-6.72150647e-01  1.89371399e+00  8.43432183e+00]
 [-1.09165114e+00  1.53881438e+00  1.27215964e+01]
 [-1.22748245e+00 -2.98174323e-02  1.40909241e+01]
 [-1.27406944e+00 -5.24248208e-01  1.45118549e+01]
 [-1.00875597e+00  4.68162311e+00  1.20690165e+01]]
---------
[[ -2.91475469   0.82381387  17.51207851]
 [-10.880315   -35.50393187  57.99803556]
 [ -2.00770984   2.89178545  12.45231184]
 [ -1.71384355  -0.25775244  10.6310936 ]
 [ -1.95120619   4.30761536  12.08749577]
 [ -1.95120619   4.30761536  12.08749577

## MVS多图